In [1]:
import pandas as pd
import os
import zipfile

# 1. Ler Arquivos .csv

In [3]:
# Sobe um nível no diretório para sair da pasta notebook
os.chdir("..")

In [4]:
# Verifica o novo diretório de trabalho atual
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [5]:
# Caminho dos diretorios
diretorio_preprocessed_I = os.path.join(diretorio_atual, "data_cvm\preprocessed_I")
os.chdir(diretorio_preprocessed_I)

In [6]:
# Lendo dados anuais
df_anual = pd.read_csv("equatorial_anual.csv")

In [7]:
# Lendo dados trimestral
df_trimestral = pd.read_csv("equatorial_trimestral.csv")

# 2. Pre-processamento anual

## 2.1 Tratando Colunas

In [8]:
df_anual[['CON_IND','TIPO_DEM']] = df_anual['GRUPO_DFP'].str.split("-", expand = True)
df_anual['CON_IND'] = df_anual['CON_IND'].str.strip()
df_anual['TIPO_DEM'] = df_anual['TIPO_DEM'].str.strip()

## 2.2 Filtros

In [9]:
# Filtrando o último exercício
df_anual = df_anual[(df_anual['ORDEM_EXERC'] == "ÚLTIMO")].copy()

In [10]:
# Demonstrativos consolidados
df_anual = df_anual[df_anual["CON_IND"] == "DF Consolidado"].copy()

## 2.3 Conversões 

In [11]:
# Converte coluna DT_REFER para o tipo datetime
df_anual['DT_REFER'] = pd.to_datetime(df_anual['DT_REFER'])
# Separa o ano e cria a coluna ano
df_anual['ANO'] = pd.DatetimeIndex(df_anual['DT_REFER']).year
# Separa o mês e cria a coluna mês
df_anual['MES'] = pd.DatetimeIndex(df_anual['DT_REFER']).month

## 2.4 Deletando Colunas

In [12]:
# Deletando colunas desnecessárias
df_anual = df_anual.drop(columns=['DT_RECEB','CD_CVM', 'ID_DOC', 'COLUNA_DF','ST_CONTA_FIXA', 'VERSAO',
                      'NUM_ITEM_PARECER_DECL', 'TXT_PARECER_DECL','COLUNA_DF','TP_RELAT_AUD',
                      'ESCALA_MOEDA', 'MOEDA','LINK_DOC','TP_PARECER_DECL', 'CATEG_DOC', 'DT_REFER',
                      'GRUPO_DFP', 'ORDEM_EXERC', 'CON_IND'
                                          ])

# 3. Pre-processamento - Trimestral

## 3.1 Tratando as Colunas

In [13]:
df_trimestral[['CON_IND','TIPO_DEM']] = df_trimestral['GRUPO_DFP'].str.split("-", expand = True)
df_trimestral['CON_IND'] = df_trimestral['CON_IND'].str.strip()
df_trimestral['TIPO_DEM'] = df_trimestral['TIPO_DEM'].str.strip()

## 3.2 Filtros

In [14]:
# Filtrando o último exercício
df_trimestral = df_trimestral[(df_trimestral['ORDEM_EXERC'] == "ÚLTIMO")].copy()

In [15]:
# Demonstrativos consolidados
df_trimestral = df_trimestral[df_trimestral["CON_IND"] == "DF Consolidado"].copy()

## 3.3 Conversões

In [16]:
# Converte coluna DT_REFER para o tipo datetime
df_trimestral['DT_REFER'] = pd.to_datetime(df_trimestral['DT_REFER'])
# Separa o ano e cria a coluna ano
df_trimestral['ANO'] = pd.DatetimeIndex(df_trimestral['DT_REFER']).year
# Separa o mês e cria a coluna mês
df_trimestral['MES'] = pd.DatetimeIndex(df_trimestral['DT_REFER']).month

## 3.4 Delentando Colunas

In [17]:
df_trimestral.columns

Index(['CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'CATEG_DOC',
       'ID_DOC', 'DT_RECEB', 'LINK_DOC', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA',
       'ORDEM_EXERC', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA',
       'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF', 'TP_RELAT_ESP',
       'TP_PARECER_DECL', 'NUM_ITEM_PARECER_DECL', 'TXT_PARECER_DECL',
       'CON_IND', 'TIPO_DEM', 'ANO', 'MES'],
      dtype='object')

In [18]:
# Deletando colunas desnecessárias
df_trimestral = df_trimestral.drop(columns=['DT_REFER', 'VERSAO', 'CD_CVM', 'CATEG_DOC',
       'ID_DOC', 'DT_RECEB', 'LINK_DOC', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA',
       'ORDEM_EXERC', 'ST_CONTA_FIXA', 'COLUNA_DF', 'TP_RELAT_ESP',
       'TP_PARECER_DECL', 'NUM_ITEM_PARECER_DECL', 'TXT_PARECER_DECL',
       'CON_IND'])

# 4. Filtrando os Demonstrativos

In [19]:
df_resultado_trimestral = df_trimestral[df_trimestral['TIPO_DEM'] == 'Demonstração do Resultado']

In [20]:
df_resultado_anual = df_anual[df_anual['TIPO_DEM'] == 'Demonstração do Resultado']

In [21]:
df_balanco_patrimonial_trimestral = df_trimestral[
                                    (df_trimestral["TIPO_DEM"] == "Balanço Patrimonial Ativo") |
                                    (df_trimestral["TIPO_DEM"] == "Balanço Patrimonial Passivo")]

In [22]:
df_balanco_patrimonial_anual = df_anual[
                                (df_anual["TIPO_DEM"] == "Balanço Patrimonial Ativo") |
                                (df_anual["TIPO_DEM"] == "Balanço Patrimonial Passivo")]

In [23]:
df_fluxo_de_caixa_trimestral = df_trimestral[(df_trimestral["TIPO_DEM"] == "Demonstração do Fluxo de Caixa (Método Indireto)")]

In [24]:
df_fluxo_de_caixa_anual = df_anual[(df_anual["TIPO_DEM"] == "Demonstração do Fluxo de Caixa (Método Indireto)")]

# 5. Exportando Arquivos

## 5.1 Diretorios

In [31]:
# Verifica o diretorio Atual
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [30]:
# Caminho dos diretorios
diretorio_processed_II = os.path.join(diretorio_atual, "data_cvm\preprocessed_II")
os.chdir(diretorio_processed_II)

## 5.2 Exportando

In [32]:
df_resultado_trimestral.to_csv('equatorial_resultado_trim.csv', index=False)
df_resultado_anual.to_csv('equatorial_resultado_anual.csv', index=False)
df_balanco_patrimonial_trimestral.to_csv('equatorial_balanco_trim.csv', index=False)
df_balanco_patrimonial_anual.to_csv('equatorial_balanco_anual.csv', index=False)
df_fluxo_de_caixa_trimestral.to_csv('equatorial_fluxo_caixa_trim.csv', index=False)
df_fluxo_de_caixa_anual.to_csv('equatorial_fluxo_caixa_anual.csv', index=False)